In [15]:
import time                   # To time processes
import warnings               # To suppress warnings

import numpy as np            # Data manipulation
import pandas as pd           # Dataframe manipulatio 
import matplotlib.pyplot as plt                   # For graphics
import seaborn as sns
import plotly as py #For World Map
py.plotly.sign_in(username='aman54kumar', api_key='bXn5Y0WroZYH8CQxKxKJ')

import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
from sklearn.preprocessing import StandardScaler  # For scaling dataset
from sklearn.cluster import KMeans, AgglomerativeClustering, AffinityPropagation #For clustering
from sklearn.mixture import GaussianMixture #For GMM clustering

import os                     # For os related operations
import sys   
import datetime
import wbdata

<script>requirejs.config({paths: { 'plotly': ['https://cdn.plot.ly/plotly-latest.min']},});if(!window.Plotly) {{require(['plotly'],function(plotly) {window.Plotly=plotly;});}}</script>

In [16]:
indicators = {
              'IC.LGL.CRED.XQ': 'Strength of legal rights index',
              'SL.EMP.WORK.ZS': 'Wage and salaried workers, total (% of total employment)',
              'SL.EMP.VULN.ZS': 'Vulnerable employment, total (% of total employment)',
              'NY.GDP.PCAP.CD': 'GDP per capita (current US$)'  
}
# 'per_lm_alllm.cov_pop_tot': 'Coverage of unemployment benefits and ALMP',
#               'per_si_allsi.cov_pop_tot': 'Coverage of social insurance programs',
#               'per_allsp.adq_pop_tot': 'Adequacy of social protection and labor programs',
#               'per_sa_allsa.adq_pop_tot': 'Adequacy of social safety net programs',
#                 'SL.TLF.0714.ZS': 'Children in employment, total (% of children ages 7-14)',
# 'SL.ISV.IFRM.ZS': 'Informal employment (% of total non-agricultural employment)',

In [17]:
data_dates = datetime.datetime(2016,1,1)
# data_dates = (datetime.datetime(2013,1,1), datetime.datetime(2017,1,1))
df = wbdata.get_dataframe(indicators, country='all', convert_date=True, data_date=data_dates)
# df = df.iloc[::-1]
df.fillna(0, inplace=True)
df = df.iloc[48:]
df = df[(df[['GDP per capita (current US$)','Strength of legal rights index',
             'Wage and salaried workers, total (% of total employment)',
             'Vulnerable employment, total (% of total employment)']] != 0).all(axis=1)]
df_real = df.copy()
df['Strength of legal rights index'] = df['Strength of legal rights index'] * 100 / 12

In [18]:
for col in df.columns:
    df[col] = df[col]/ df[col].sum()

  
df['decent_work_score'] = (df[df.columns[0]] + df[df.columns[1]] + df[df.columns[2]] + df[df.columns[3]])*100
df_real['decent_work_score'] = df['decent_work_score']
df_real.to_csv(r'wbdata.csv')
print(df.head())
df_real.head()

           Strength of legal rights index  \
country                                     
Albania                          0.006704   
Algeria                          0.002235   
Angola                           0.001117   
Argentina                        0.003352   
Armenia                          0.006704   

           Wage and salaried workers, total (% of total employment)  \
country                                                               
Albania                                             0.004108          
Algeria                                             0.006997          
Angola                                              0.002888          
Argentina                                           0.007590          
Armenia                                             0.005953          

           Vulnerable employment, total (% of total employment)  \
country                                                           
Albania                                             

,Strength of legal rights index,"Wage and salaried workers, total (% of total employment)","Vulnerable employment, total (% of total employment)",GDP per capita (current US$),decent_work_score
country,,,,,
Albania,6.0,40.568001,56.997002,4131.872341,2.208717
Algeria,2.0,69.095001,27.825999,3916.881571,1.559626
Angola,1.0,28.518000,66.547998,3308.772828,1.648368
Argentina,3.0,74.948997,21.112999,12654.354999,2.015317
Armenia,6.0,58.785000,40.328000,3605.743117,2.094348


In [19]:
print("Dimension of dataset:", df.shape)
df.dtypes

Dimension of dataset: (165, 5)


Strength of legal rights index                              float64
Wage and salaried workers, total (% of total employment)    float64
Vulnerable employment, total (% of total employment)        float64
GDP per capita (current US$)                                float64
decent_work_score                                           float64
dtype: object

In [20]:
df1 = df[['Strength of legal rights index', 'Wage and salaried workers, total (% of total employment)', 
         'Vulnerable employment, total (% of total employment)','GDP per capita (current US$)']]
cor = df1.corr()
sns.heatmap(cor, square = True)

In [21]:
df2 = df.reset_index()
data = dict(type='choropleth',
           locations=df2['country'],
           locationmode = 'country names',
           z = df2['decent_work_score'],
           text = df2['country'],
           colorbar = {'title':  'Decent work'})
layout = dict(title = 'Decent Work Index',
            geo = dict(showframe= False,
                      projection = {'type': 'mercator'}))

choromap3 = go.Figure(data = [data], layout=layout)
iplot(choromap3)
#show map

<div id="964c286a-662c-44c3-89cf-73aebd41fca2" style="height: 525px; width: 100%;" class="plotly-graph-div"></div><script type="text/javascript">require(["plotly"], function(Plotly) { window.PLOTLYENV=window.PLOTLYENV || {};window.PLOTLYENV.BASE_URL="https://plot.ly";Plotly.newPlot("964c286a-662c-44c3-89cf-73aebd41fca2", [{"colorbar": {"title": "Decent work"}, "locationmode": "country names", "locations": ["Albania", "Algeria", "Angola", "Argentina", "Armenia", "Australia", "Austria", "Azerbaijan", "Bahamas, The", "Bahrain", "Bangladesh", "Barbados", "Belarus", "Belgium", "Belize", "Benin", "Bhutan", "Bosnia and Herzegovina", "Botswana", "Brazil", "Brunei Darussalam", "Bulgaria", "Burkina Faso", "Burundi", "Cabo Verde", "Cambodia", "Cameroon", "Canada", "Central African Republic", "Chad", "Chile", "China", "Colombia", "Comoros", "Congo, Dem. Rep.", "Congo, Rep.", "Costa Rica", "Cote d'Ivoire", "Croatia", "Cyprus", "Czech Republic", "Denmark", "Djibouti", "Dominican Republic", "Ecuador", "Egypt, Arab Rep.", "El Salvador", "Equatorial Guinea", "Estonia", "Eswatini", "Ethiopia", "Fiji", "Finland", "France", "Gabon", "Gambia, The", "Georgia", "Germany", "Ghana", "Greece", "Guatemala", "Guinea", "Guinea-Bissau", "Guyana", "Haiti", "Honduras", "Hong Kong SAR, China", "Hungary", "Iceland", "India", "Indonesia", "Iran, Islamic Rep.", "Ireland", "Israel", "Italy", "Jamaica", "Japan", "Kazakhstan", "Kenya", "Korea, Rep.", "Kuwait", "Kyrgyz Republic", "Lao PDR", "Latvia", "Lebanon", "Lesotho", "Liberia", "Lithuania", "Luxembourg", "Macedonia, FYR", "Madagascar", "Malawi", "Malaysia", "Maldives", "Mali", "Malta", "Mauritania", "Mauritius", "Mexico", "Moldova", "Mongolia", "Montenegro", "Morocco", "Mozambique", "Myanmar", "Namibia", "Nepal", "Netherlands", "New Zealand", "Nicaragua", "Niger", "Nigeria", "Norway", "Oman", "Pakistan", "Panama", "Papua New Guinea", "Paraguay", "Peru", "Philippines", "Poland", "Portugal", "Puerto Rico", "Qatar", "Romania", "Russian Federation", "Rwanda", "Samoa", "Saudi Arabia", "Senegal", "Serbia", "Sierra Leone", "Singapore", "Slovak Republic", "Slovenia", "Solomon Islands", "South Africa", "South Sudan", "Spain", "Sri Lanka", "St. Lucia", "St. Vincent and the Grenadines", "Sudan", "Suriname", "Sweden", "Switzerland", "Tajikistan", "Tanzania", "Thailand", "Togo", "Tonga", "Trinidad and Tobago", "Tunisia", "Turkey", "Uganda", "Ukraine", "United Arab Emirates", "United Kingdom", "United States", "Uruguay", "Uzbekistan", "Vanuatu", "Vietnam", "Zambia", "Zimbabwe"], "text": ["Albania", "Algeria", "Angola", "Argentina", "Armenia", "Australia", "Austria", "Azerbaijan", "Bahamas, The", "Bahrain", "Bangladesh", "Barbados", "Belarus", "Belgium", "Belize", "Benin", "Bhutan", "Bosnia and Herzegovina", "Botswana", "Brazil", "Brunei Darussalam", "Bulgaria", "Burkina Faso", "Burundi", "Cabo Verde", "Cambodia", "Cameroon", "Canada", "Central African Republic", "Chad", "Chile", "China", "Colombia", "Comoros", "Congo, Dem. Rep.", "Congo, Rep.", "Costa Rica", "Cote d'Ivoire", "Croatia", "Cyprus", "Czech Republic", "Denmark", "Djibouti", "Dominican Republic", "Ecuador", "Egypt, Arab Rep.", "El Salvador", "Equatorial Guinea", "Estonia", "Eswatini", "Ethiopia", "Fiji", "Finland", "France", "Gabon", "Gambia, The", "Georgia", "Germany", "Ghana", "Greece", "Guatemala", "Guinea", "Guinea-Bissau", "Guyana", "Haiti", "Honduras", "Hong Kong SAR, China", "Hungary", "Iceland", "India", "Indonesia", "Iran, Islamic Rep.", "Ireland", "Israel", "Italy", "Jamaica", "Japan", "Kazakhstan", "Kenya", "Korea, Rep.", "Kuwait", "Kyrgyz Republic", "Lao PDR", "Latvia", "Lebanon", "Lesotho", "Liberia", "Lithuania", "Luxembourg", "Macedonia, FYR", "Madagascar", "Malawi", "Malaysia", "Maldives", "Mali", "Malta", "Mauritania", "Mauritius", "Mexico", "Moldova", "Mongolia", "Montenegro", "Morocco", "Mozambique", "Myanmar", "Namibia", "Nepal", "Netherlands", "New Zealand", "Nicaragua", "Niger", "Nigeria", "Norway", "Oman", "Pakistan", "Panama", "Papua New Gu

In general, k-means is the first choice for clustering because of its simplicity. Here, the user has to define the number of clusters. The clusters are formed based on the closeness to the center value of the clusters. The initial center value is chosen randomly. K-means clustering is top-down approach, in the sense, we decide the number of clusters (k) and then group the data points into k clusters.

In [22]:
def doKmeans(X, nclust=2):
    model = KMeans(nclust)
    model.fit(X)
    clust_labels = model.predict(X)
    cent = model.cluster_centers_
    return (clust_labels, cent)

clust_labels, cent = doKmeans(df_real, 2)
kmeans = pd.DataFrame(clust_labels)
df_real.insert((df_real.shape[1]),'kmeans',kmeans)

In [23]:
fig = plt.figure()
ax = fig.add_subplot(111)
scatter = ax.scatter(df_real['GDP per capita (current US$)'],df_real['decent_work_score'],
                     c=kmeans[0], s=50)
ax.set_title('K-Means Clustering')
ax.set_xlabel('GDP per Capita')
ax.set_ylabel('Decent Work')
plt.colorbar(scatter)